In [1]:
import numpy as np
from matplotlib import pyplot as plt
from flatsurf import *
import os
from Poincare import *
from surface_dynamics.all import *

# import pandas as pd
# import pwlf
# import sympy as sym
# from sympy import Symbol, solve, lambdify
# from Poincare import Section
# import math
# from time import time

In [2]:
# Code from Sunrose
# Gives non-visibility tori for testing
D = OrigamiDatabase()
q = D.query()
qlist = q.list()


def unit_hor_saddle(O):
    count = 0
    for vert in O.vertices():
        tup = vert.up_right_tuple()
        for i in tup:
            for vert2 in O.vertices():
                tup2 = vert2.up_right_tuple()
                if O.r()(i) in tup2:
                    return True
    return False


def is_unobstructed(O):
    cusp_reps = O.teichmueller_curve().cusp_representatives()
    for item in cusp_reps:
        if not unit_hor_saddle(item[0]):
            return False
    return True


def obstructed(n, **kwargs):
    obstructed = []
    count_obstructed = 0
    p = D.query(nb_squares=n, **kwargs)
    for item in p:
        if not is_unobstructed(item):
            obstructed.append(item)
            count_obstructed += item.teichmueller_curve().orbit_graph().num_verts()
    return (obstructed, count_obstructed)

In [3]:
# generate vectors for saddle connections on STS
def vectors(perm, length=200):
    a = str(perm)
    h, v = a.split("\n")
    S = SymmetricGroup(len(h))
    T = translation_surfaces.origami(S(h), S(v))
    T = T.erase_marked_points()
    sc_list = T.saddle_connections(length)
    slopes_all = []
    for item in sc_list:
        vec = item.holonomy().n()
        direction = item.direction
        if vec not in slopes_all:
            if vec[0] >= -length/20 and vec[0] <= length/20:
                if vec[1] >= -length/20 and vec[1] <= length/20:
                    slopes_all.append(item.holonomy().n())
    vecs = []
    for vec in slopes_all:
        item = np.array([[vec[0]], [vec[1]]])
        vecs.append(item)
    return vecs

In [4]:
# list of permutations
def perms_list(n, **kwargs):
    obstructed = []
    p = D.query(nb_squares=n, **kwargs)
    for item in p:
        if not is_unobstructed(item):
            obstructed.append(item)
            for perm in item.teichmueller_curve():
                obstructed.append(perm)
    return obstructed

In [5]:
# take files from saddle.py and load them into notebook
def load_arrays_from_file(file_path):
    # Load arrays from the NumPy file
    arrays_list = np.load(file_path, allow_pickle=True)

    # Ensure each element in the list is a NumPy array
    arrays_list = [np.array(array) for array in arrays_list]

    return arrays_list

In [6]:
# get list of permutations for STS with 7 squares
permutations = perms_list(7)
len(permutations)

92

In [7]:
perm = permutations[4]

In [8]:
vecs0 = load_arrays_from_file(os.path.join("vecs", "vecs7-4.npy"))
vecs1 = load_arrays_from_file(os.path.join("vecs", "vecs7-3.npy"))

In [9]:
len(vecs0)

3618904

In [10]:
len(vecs1)

907654

In [11]:
# load these values after kernel crashes
%store -r a
%store -r c
%store -r e
%store -r g

In [12]:
len(a)

66

In [ ]:
# Run computations for individual cusps and see if the output is decent
# j represents the jth cusps
# change i to get new output for the jth cusp

# works: 0, 24, ?, ?, ?, ?, ?, ?, ?, 1
i = 47
j = 3
# maybe 0
n_squares = 7
dx = 0.01
index = 4
vecs, x_vals, m0, m1, x0, y0, dx_y = setup(
    a[i][j], c[i][j], e[i][j], vecs0, dx)
df = winners(vecs, x_vals, m0, m1, y0, dx, dx_y)

# output is:
# number of vectors
# the x and y coords of the "section vector"
# poincare section plot

3618904
38.46910948952175 0.0006766825527409152


In [ ]:
#267.72

#plot the poincare section for the cusp represenation
plot(df, vecs, c[i][j], j, n_squares, index, test = false)

In [ ]:
# plot the pdf for this cusp
sec_list = sec_setup(df, dx_y)
secs = sec_comp(sec_list, dx)
times = time_comp(secs)
pdf(list(df["time"]), times, dx)
plt.savefig(os.path.join("results", f"{n_squares} - {index}", f"pdf {j}"))

In [ ]:
# For an STS, repeatedly run the poincare_details function and get the
# different representations of cusps and other values and store them in lists
# takes awhile
a = []
c = []
e = []
g = []
for i in range(100):
    try:
        alphas, Cs, C_invs, eigs, Ms, generators, eigenvecs = poincare_details(
            perm, vecs0)
    except:
        continue
    a.append(alphas)
    c.append(Cs)
    e.append(eigenvecs)
    g.append(generators)

In [ ]:
# store these values in case kernel crashes
%store a
%store c
%store e
%store g